<h1><center>Selectivity Analysis in Medium Voltage Distribution Networks</center></h1>
<p>Author: Yuri Perim</p>

<h3>Modules</h3>

In [ ]:
import os
from pathlib import Path
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import py_dss_interface
import net_utils
import prot_utils
import sc_utils

<h3>OpenDSS</h3>

In [ ]:
dss = py_dss_interface.DSSDLL()

<h3>Directory and Files</h3>

In [ ]:
DIR = Path(r'C:\Users\yurip\GitHub\Public\EDS_Protection\8500-Node')

MASTER = 'Master-unbal-python.dss'
dss_file = os.path.join(DIR, MASTER)

TCC_CURVES = 'TCC_Curve.dss'

FUSES = 'Fuses.DSS'
RECLOSERS = 'Reclosers.DSS'

<h3>Network Graph</h3>

In [ ]:
G = net_utils.ckt_build_graph(dss, dss_file)

<h3>TCC Curves</h3>

In [ ]:
tcc_dict = prot_utils.get_tcc_curves(dss, dss_file, TCC_CURVES)

<h3>Protection Graph</h3>

In [ ]:
pdelt_fuse_dict = net_utils.get_fuses(dss, dss_file, TCC_CURVES, FUSES)
pdelt_recloser_dict = net_utils.get_reclosers(dss, dss_file, TCC_CURVES, RECLOSERS)
pdelt_ctrlelt_dict = {**pdelt_fuse_dict, **pdelt_recloser_dict}

P = net_utils.prot_build_graph(G, pdelt_ctrlelt_dict)

if nx.is_branching(P):
    if nx.is_arborescence(P):
        display('Way to go')
    else:
        display('Almost there')
else:
    display('Verify')

<h3>Analysis</h3>

In [ ]:
#voltage_bases = [13.8]
voltage_bases = [115, 12.47, 0.48, 0.208]

root = set([ctrlelt for ctrlelt in P.nodes() if P.in_degree(ctrlelt) == 0]).pop()

threshold = 0.2

for outer_node in P.nodes():
    sc_bus = P.nodes[outer_node]['sc_bus']
    sc_utils.sc_sim(dss, dss_file, bus_name=sc_bus, sc_type='1001', volt_list=voltage_bases, max_iter=30)
    
    path = nx.shortest_path(P, source=root, target=outer_node)[::-1]
    current_time_list = []
    for inner_node in path:
        prot_node_dict = {inner_node: P.nodes[inner_node]}
        currents_dict = sc_utils.get_elt_currents(dss, prot_node_dict[inner_node]['pdelt'])
        current, time = prot_utils.compute_time(prot_node_dict, pdelt_ctrlelt_dict, tcc_dict, currents_dict)
        current_time_list.append((inner_node, current, time))
    
    for i in range(len(current_time_list) - 1):
        actual_prot, actual_current, actual_time = current_time_list[i]
        upstream_prot, upstream_current, upstream_time = current_time_list[i + 1]
        
        if actual_time == float('Inf') or upstream_time == float('Inf'):
            pass
        else:
            delta_t = upstream_time - actual_time
            if delta_t < threshold:
                display(f'SLG fault on bus {sc_bus}:')
                display(f'\u2022 {actual_prot} is not selective with upstream {upstream_prot}')
                display(f'\u2022 {actual_prot} -> current: {actual_current:.4f}, time: {actual_time:.6f} | {upstream_prot} -> current: {upstream_current:.4f}, time: {upstream_time:.6f}')
                display(f'\u2022 Time delta = {delta_t:.6f}')

<h3>Coordination Graph</h3>

In [ ]:
devices = ['fuse.ln5744326-1', 'fuse.ln6229831-1', 'recloser.subbreaker']

c_array = np.logspace(1, 3, num=30, endpoint=True, base=10.0)

fig, ax = plt.subplots()

for pdelt, ctrlelt_dict in pdelt_ctrlelt_dict.items():
    ctrlelt = ctrlelt_dict['name']
    if ctrlelt in devices:
        prot_node_dict = {ctrlelt: P.nodes[ctrlelt]}
        ph_tcc_func, gr_tcc_func = prot_utils.get_ctrlelt_tcc_func(prot_node_dict, pdelt_ctrlelt_dict, tcc_dict)
        ax.loglog(c_array, gr_tcc_func(c_array), label=ctrlelt)

ax.grid(True, which='both')
ax.set_xlabel('Current')
ax.set_ylabel('Time')
ax.legend()

plt.show()

<h3>Sandbox</h3>

In [ ]:
nx.draw(P)